## LLM

In [34]:
import tensorflow as tf
import numpy as np
import re

In [2]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

In [31]:
num_train = len(ds["train"])

In [ ]:
text = ""

for i in range(1000):
    text += ds["train"][i]["text"]
    text += " "

In [ ]:
#standardize the text

text_clean = re.sub('â€', '"', text)
text_clean = re.sub('â€œ', '"', text_clean)
text_clean = re.sub('œ', '', text_clean)

text_clean = re.sub("™", "'", text_clean)

#clean the text

text_clean = re.sub(',', '', text_clean)
text_clean = re.sub('"', '', text_clean)
text_clean = re.sub('œ', '', text_clean)
text_clean = re.sub('!', '', text_clean)
text_clean = re.sub('\?', '', text_clean)
text_clean = re.sub("™", "'", text_clean)
text_clean = re.sub("'s", '', text_clean)
text_clean = re.sub('\n', ' ', text_clean)
text_clean = re.sub("\.", '', text_clean)


In [177]:
text_sep = (text_clean.lower()).split(" ")
print(len(text_sep))
text_sep = list(set(text_sep))
print(len(text_sep))

189392
4676


In [178]:
text_sep

['',
 'went',
 'strong',
 'eat',
 'intelligent',
 'marching',
 'elephant',
 'shh',
 'streamers',
 'meet',
 'future',
 'rust',
 'burps',
 'sword',
 'wasted',
 'bag',
 'heel',
 'although',
 'three-year-old',
 'cuddling',
 'clink',
 'welcomed',
 'became',
 'dreaming',
 'sailing',
 'plastic',
 'cartoon',
 "couldn't",
 'windy',
 'rafts',
 'thinks:',
 'how',
 'uh-oh',
 'popsicle',
 'dust',
 'he',
 'pants',
 'limits',
 'containers',
 'bug',
 'would',
 'brave',
 'climb',
 'dictionary',
 'grown',
 'film',
 'towels',
 'sipped',
 'near',
 'visitor',
 'hopes',
 'sang',
 'addition',
 'elderly',
 'such',
 'mary',
 'serve',
 'catch',
 'valuable',
 'attracting',
 'toes',
 "'that",
 'trumpet',
 'awestruck',
 'pillow',
 'sniffles',
 'cube',
 'trash',
 'activity',
 'think',
 'cut',
 'tell',
 'upper',
 'barrel',
 'instinctively',
 'depths',
 'hooting',
 'exactly',
 'preferences',
 'gold',
 'darker',
 'ideas',
 'flower',
 'feeding',
 'cushions',
 'terrified',
 'pancakes',
 'rearranged',
 'race',
 'will',
 

## Token Stream

In [33]:
vocab_size = 10
length = 20
np.random.seed(42)

# Create a random token stream
x_stream = np.random.randint(0, vocab_size, size=(length,))

## Embedding Matrix Key and Query

In [34]:
tf.random.set_seed(42)

embedding_dim = 8
embedding_matrix = tf.Variable(tf.random.uniform([vocab_size, embedding_dim], -1, 1))


key_dim = 4
quary_dim = 4
value_dim = 4

WK =  tf.Variable(tf.random.uniform([key_dim, embedding_dim], -1, 1))
WQ =  tf.Variable(tf.random.uniform([quary_dim, embedding_dim], -1, 1))
WV =  tf.Variable(tf.random.uniform([value_dim, embedding_dim], -1, 1))

## Select from word matrix

In [76]:
x_embeds = tf.nn.embedding_lookup(embedding_matrix, x_stream)

x_k = tf.linalg.matvec(WK, x_embeds)
x_q = tf.linalg.matvec(WQ, x_embeds)
x_v = tf.linalg.matvec(WV, x_embeds)

inner = tf.linalg.matmul(x_k, x_q, transpose_b=True)
mask = tf.linalg.band_part(tf.ones((length, length), dtype = tf.bool), 0, -1)
inner_masked = tf.where(mask, inner, tf.constant(-np.inf))

dk = tf.sqrt(tf.cast(key_dim, tf.float32))
WA = tf.nn.softmax(inner_masked/dk, axis=-1)

In [85]:
head = x_v + WA @ x_v

## 

In [ ]:
# dense layer

dense_layer1 = tf.keras.layers.Dense(20, activation='relu')
dense_layer2 = tf.keras.layers.Dense(vocab_size, activation='relu')

x = dense_layer1(head)
x = dense_layer2(x)

output = tf.nn.softmax(x)

In [90]:
print(output.shape)

(20, 10)


ModuleNotFoundError: No module named 'datasets'